In [1]:
## torch.nn은 신경망을 구성하는 데 필요한 모든 요소 제공
## 모든 모듈은 nn.Module의 하위 클래스임
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = ('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device} device')

Using cpu device


In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() ## flatten의 시작은 무조건 dim = 1부터 마지막까지!
        ## 출력은 (n x (dim1 * dim2 * ---))
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [19]:
X = torch.rand(1, 28, 28, device = device)
logits = model(X)
pred_probab = nn.Softmax(dim = 1)(logits)
predict = pred_probab.argmax(1)
print(f'Predicted class : {predict}')

Predicted class : tensor([4])


In [11]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [20]:
layer1 = nn.Linear(in_features = 28 * 28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [21]:
print(f'Before ReLU : {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f'After ReLU : {hidden1}')

Before ReLU : tensor([[-0.2681,  0.2549,  0.0067, -0.1369,  0.1084, -0.1936,  0.1234, -0.0788,
         -0.4158,  0.3037,  0.0917,  0.1030, -0.4199, -0.4003,  0.1087, -0.3017,
          0.1742,  0.2533, -0.0754,  0.5195],
        [ 0.1332,  0.4467,  0.0275, -0.0221,  0.3425, -0.1886,  0.0616, -0.0014,
         -0.3089,  0.4018,  0.1954, -0.2551, -0.2066, -0.7803,  0.7569,  0.0798,
         -0.0864,  0.0528,  0.1863,  0.2949],
        [ 0.4177,  0.2012, -0.0823, -0.0705,  0.0620, -0.3529,  0.2553,  0.2156,
         -0.2437,  0.1600,  0.2365,  0.0098, -0.0691, -0.3066,  0.1798,  0.4237,
         -0.2720,  0.0408, -0.0446,  0.3188]], grad_fn=<AddmmBackward0>)


After ReLU : tensor([[0.0000, 0.2549, 0.0067, 0.0000, 0.1084, 0.0000, 0.1234, 0.0000, 0.0000,
         0.3037, 0.0917, 0.1030, 0.0000, 0.0000, 0.1087, 0.0000, 0.1742, 0.2533,
         0.0000, 0.5195],
        [0.1332, 0.4467, 0.0275, 0.0000, 0.3425, 0.0000, 0.0616, 0.0000, 0.0000,
         0.4018, 0.1954, 0.0000, 0.0000, 0.0000, 0.

In [22]:
seq_modules= nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [25]:
## 모델 매개변수 -> 학습 중에 최적화되는 가중치와 편향
## nn.Module 상속하면 모델 개체 내부의 모든 필드들이 자동으로 추적
print(f'Model structure : {model}\n\n')

for name, param in model.named_parameters():
    print(f'Layer : {name} | Size : {param.size()} | Values : {param[:2]} \n')

Model structure : NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer : linear_relu_stack.0.weight | Size : torch.Size([512, 784]) | Values : tensor([[-0.0300,  0.0288,  0.0176,  ..., -0.0303,  0.0037, -0.0089],
        [ 0.0175,  0.0278, -0.0294,  ...,  0.0024,  0.0097, -0.0251]],
       grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.0.bias | Size : torch.Size([512]) | Values : tensor([-0.0234,  0.0200], grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2.weight | Size : torch.Size([512, 512]) | Values : tensor([[-0.0385,  0.0206, -0.0345,  ...,  0.0294,  0.0292, -0.0110],
        [ 0.0155,  0.0251, -0.0117,  ...,  0.0100,  0.0178, -0.0322]],
       grad_fn=<SliceBackward0>) 

Layer : linear_relu_stack.2